In [ ]:
# !git clone https://github.com/shivangi1299/capstone.git
!git clone https://github.com/prosis369/capstone.git

In [ ]:
%cd capstone

In [ ]:
!pip3 install pytorch_pretrained_bert
!pip3 install numpy
!pip3 install nltk
!pip3 install torch
!pip3 install pandas
!pip3 install sklearn
!pip3 install matplotlib

In [ ]:
!pip3 install transformers

In [ ]:
!pip3 install sklearn

In [ ]:
import nltk
nltk.download('stopwords')

To run a pytorch bias model - need not run this for training transfer learning model

In [ ]:
!python3 transfer_bias.py

TRANSFER LEARNING FOR BIAS BEGINS

Toxic Dataset

In [18]:
import numpy as np
import torch
from torch import nn
from torch import optim
from utils import np2autograd
from torch.autograd import Variable
from torch.nn import functional as F
from utils import max_sentence_size
from utils import avg_cross_entropy_loss
from utils import batch_generator_bias
from sklearn.metrics import accuracy_score

In [19]:
# Hyperparams
postag_reg = 1e-3
chunking_reg = 1e-3
learning_rate = 1e-3
postag_hn_size = 100
postag_nb_layers = 2
embedding_size = 512
bias_reg = 1e-3
nb_postags = 1

In [20]:
class TransferBiasClassification(nn.Module):
  
    def __init__(self):
        super(TransferBiasClassification, self).__init__()

        self.w = nn.Parameter(torch.randn(postag_nb_layers * 2,
                                          max_sentence_size,
                                          postag_hn_size))
        self.h = nn.Parameter(torch.randn(postag_nb_layers * 2,
                                          max_sentence_size,
                                          postag_hn_size))

        # Bidirectional LSTM
        self.bi_lstm = nn.LSTM(embedding_size,
                               postag_hn_size,
                               postag_nb_layers,
                               bidirectional=True)

        self.fc = nn.Linear(postag_hn_size * 2, nb_postags)

    def forward(self, x):
        # Runs the LSTM for each word-vector in the sentence x
        # x = [x]
        x = torch.tensor(x, dtype=torch.float32)
        out, hn = self.bi_lstm(x, (self.h[:, :x.size(1), :],
                                   self.w[:, :x.size(1), :]))

        # Runs a linear classifier on the outputed state vector
        tags = self.fc(out[0])

        return tags
    
    def bias_loss(self, y, yt):
        loss = (yt.float() - y) ** 2 \
               + (transfer_bias_model.w.norm() ** 2) * bias_reg

        return loss

    def loss(self, y, bias):
        # print("Hi i am in loss")
        losses = []
        # print("Hi i am in loss line 2")
        length = len(y)

        for i in range(length):

            p_bias, r_bias = y[i][0], np2autograd(bias[i])
            loss_bias = self.bias_loss(p_bias, r_bias)

            losses.append(loss_bias)

        loss = losses[0]

        for i in range(1, length):
            loss += losses[i]

        loss = loss / length

        return loss


def threshold(prediction, upperBound, lowerBound):
  # print(type(prediction))
  train_predictions.append(prediction)
  if prediction >= upperBound:
      prediction = 0
  elif prediction < lowerBound:
      prediction = 0
  else:
      prediction = 1
  return prediction

def compare(out):

    # print(out[0][0].numpy())
    predicted_bias = out[0][0].numpy()
    predicted_bias = threshold(predicted_bias, 0.2, 0.1)
    
    return [predicted_bias]

def accuracy(train_batch_acc, bias_nb_batches):

  pred_bias = []

  l = len(train_batch_acc)

  for i in range(l):
    pred_bias.append(train_batch_acc[i][0])

  print(pred_bias)
  print(bias_nb_batches)

  bias_acc = accuracy_score(bias_nb_batches, pred_bias)
  
  return(bias_acc)

def mapPrediction(predicted_bias, bias_nb_batches):
  # print(type(predicted_bias))
  # predicted_bias = predicted_bias.astype(np.float)
  # print(predicted_bias)
  actual_0 = []
  actual_1 = []

  for i in range(len(predicted_bias)):

    # print(type(predicted_bias[i]))
    # print(type(bias_nb_batches[i]))
    if len(actual_0)==0 and len(actual_1)==0:
      if bias_nb_batches[i] == 0:
        actual_0.append(predicted_bias[i])
      else:
        actual_1.append(predicted_bias[i])
    else:
      if bias_nb_batches[i] == 0:
        # actual_0[0].append(predicted_bias[i])
        actual_0[0] = np.append(actual_0[0], predicted_bias[i])
      else:
        # actual_1[0] = np.append(actual_1[0], predicted_bias[i])
        actual_1.append(predicted_bias[i])
  
  # print(type(actual_0[0]))
  print(actual_0[0].sort())
  print(actual_1)


In [21]:
dataset_file = './toxic_train_30k.csv'

In [22]:
nb_epochs = 1
# batch_size = 47
batch_size = 1
nb_batches = 1000

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gen = batch_generator_bias(batch_size, nb_batches, dataset_file)
transfer_bias_model = TransferBiasClassification()
# transfer_bias_model = transfer_bias_model.to(device)
adam = optim.Adam(transfer_bias_model.parameters(), lr=learning_rate)

train_epoch_acc = []

PATH = "saved_model_bias_onlyToxic1k.pt"

In [23]:
train_predictions = []

In [24]:
for epoch in range(nb_epochs):

    train_batch_loss = []
    train_batch_acc = []

    transfer_bias_nb_batches = []

    for batch in range(nb_batches):

        text, transfer_bias = next(gen)

        transfer_bias_nb_batches.append(transfer_bias[0][0])
        
        out = transfer_bias_model.forward(text)

        loss = transfer_bias_model.loss(out, transfer_bias)
        
        # out = transfer_bias_model.forward(text).to(device)

        # loss = transfer_bias_model.loss(out, transfer_bias).to(device)
        print("Epoch:", epoch,
              "Batch:", batch,
              "Loss:", loss.data[0])

        adam.zero_grad()
        # loss.backward()
        loss.sum().backward()
        adam.step()
        # print("out", out)

        torch.save(transfer_bias_model.state_dict(), PATH)


        transfer_bias_model.eval() # enter evaluation mode
        with torch.no_grad():
              train_batch_acc.append(compare(out)) # evaluate mini-batch train accuracy in evaluation

    mapPrediction(train_predictions, transfer_bias_nb_batches)
    acc = accuracy(train_batch_acc, transfer_bias_nb_batches)
    print("Epoch: ", epoch, "Bias Accuracy: ", acc)

Epoch: 0 Batch: 0 Loss: tensor(120.0497)
Epoch: 0 Batch: 1 Loss: tensor(119.8824)
Epoch: 0 Batch: 2 Loss: tensor(119.6653)
Epoch: 0 Batch: 3 Loss: tensor(119.4735)
Epoch: 0 Batch: 4 Loss: tensor(119.3098)
Epoch: 0 Batch: 5 Loss: tensor(119.1010)
Epoch: 0 Batch: 6 Loss: tensor(120.1235)
Epoch: 0 Batch: 7 Loss: tensor(118.7439)
Epoch: 0 Batch: 8 Loss: tensor(118.5689)
Epoch: 0 Batch: 9 Loss: tensor(118.3641)
Epoch: 0 Batch: 10 Loss: tensor(118.2017)
Epoch: 0 Batch: 11 Loss: tensor(118.0292)
Epoch: 0 Batch: 12 Loss: tensor(118.3523)
Epoch: 0 Batch: 13 Loss: tensor(117.6207)
Epoch: 0 Batch: 14 Loss: tensor(117.4890)
Epoch: 0 Batch: 15 Loss: tensor(117.3066)
Epoch: 0 Batch: 16 Loss: tensor(117.4569)
Epoch: 0 Batch: 17 Loss: tensor(116.8978)
Epoch: 0 Batch: 18 Loss: tensor(116.7792)
Epoch: 0 Batch: 19 Loss: tensor(116.5541)
Epoch: 0 Batch: 20 Loss: tensor(116.3680)
Epoch: 0 Batch: 21 Loss: tensor(116.1680)
Epoch: 0 Batch: 22 Loss: tensor(115.9027)
Epoch: 0 Batch: 23 Loss: tensor(115.7202)
Ep

Olid Dataset

In [25]:
dataset_file = './olid-train.csv'

In [26]:
nb_epochs = 1
# batch_size = 47
batch_size = 1
nb_batches = 1000
# 13240

gen = batch_generator_bias(batch_size, nb_batches, dataset_file)
# transfer_bias_model = TransferBiasClassification()
# adam = optim.Adam(transfer_bias_model.parameters(), lr=learning_rate)

train_epoch_acc = []
PATH = "saved_model_bias_toxic30kAndOlid1k.pt"

In [27]:
for epoch in range(nb_epochs):

    train_batch_loss = []
    train_batch_acc = []

    transfer_bias_nb_batches = []

    for batch in range(nb_batches):

        text, transfer_bias = next(gen)

        transfer_bias_nb_batches.append(transfer_bias[0][0])
        
        out = transfer_bias_model.forward(text)

        loss = transfer_bias_model.loss(out, transfer_bias)
        print("Epoch:", epoch,
              "Batch:", batch,
              "Loss:", loss.data[0])

        adam.zero_grad()
        # loss.backward()
        loss.sum().backward()
        adam.step()
        # print("out", out)

        torch.save(transfer_bias_model.state_dict(), PATH)


        transfer_bias_model.eval() # enter evaluation mode
        with torch.no_grad():
              train_batch_acc.append(compare(out)) # evaluate mini-batch train accuracy in evaluation

    acc = accuracy(train_batch_acc, transfer_bias_nb_batches)
    print("Epoch: ", epoch, "Bias Accuracy: ", acc)

Epoch: 0 Batch: 0 Loss: tensor(25.4229)
Epoch: 0 Batch: 1 Loss: tensor(25.3836)
Epoch: 0 Batch: 2 Loss: tensor(25.3448)
Epoch: 0 Batch: 3 Loss: tensor(25.3022)
Epoch: 0 Batch: 4 Loss: tensor(25.2597)
Epoch: 0 Batch: 5 Loss: tensor(25.2249)
Epoch: 0 Batch: 6 Loss: tensor(25.1780)
Epoch: 0 Batch: 7 Loss: tensor(25.1428)
Epoch: 0 Batch: 8 Loss: tensor(25.1072)
Epoch: 0 Batch: 9 Loss: tensor(25.0696)
Epoch: 0 Batch: 10 Loss: tensor(25.0233)
Epoch: 0 Batch: 11 Loss: tensor(24.9831)
Epoch: 0 Batch: 12 Loss: tensor(24.9458)
Epoch: 0 Batch: 13 Loss: tensor(24.9053)
Epoch: 0 Batch: 14 Loss: tensor(24.8660)
Epoch: 0 Batch: 15 Loss: tensor(24.8304)
Epoch: 0 Batch: 16 Loss: tensor(24.7898)
Epoch: 0 Batch: 17 Loss: tensor(24.7496)
Epoch: 0 Batch: 18 Loss: tensor(24.7137)
Epoch: 0 Batch: 19 Loss: tensor(24.6735)
Epoch: 0 Batch: 20 Loss: tensor(24.6334)
Epoch: 0 Batch: 21 Loss: tensor(24.5968)
Epoch: 0 Batch: 22 Loss: tensor(24.5571)
Epoch: 0 Batch: 23 Loss: tensor(24.5183)
Epoch: 0 Batch: 24 Loss: t

In [28]:
acc = accuracy(train_batch_acc, transfer_bias_nb_batches)
print("Epoch: ", epoch, "Bias Accuracy: ", acc)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [35]:
import csv
file = open("toxic_train_30k.csv")
reader = csv.reader(file)
lines= len(list(reader))
print(lines)

30001


In [31]:
!pwd

/content/capstone
